##ENTORNO

In [1]:
#Instalar dependencias para instalar Google-Chrome
!sudo apt-get update
!sudo apt-get install -y libu2f-udev
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,748 kB]
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal/main amd64 Packages [46.7 kB]

In [2]:
#Instalar Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f
!rm -rf google-chrome-stable_current_amd64.deb

--2023-05-31 14:58:42--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 108.177.12.93, 108.177.12.91, 108.177.12.190, ...
Connecting to dl.google.com (dl.google.com)|108.177.12.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93841272 (89M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>]  89.49M  73.4MB/s    in 1.2s    

2023-05-31 14:58:43 (73.4 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [93841272/93841272]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 122662 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (114.0.5735.90-1) ...
Setting up google-chrome-stable (114.0.5735.90-1) ...
update-alternatives: using /usr/bin/google-chrome-stable to provide /usr/bin/x-www-browse

In [3]:
# Descargar WebDriver
!wget https://chromedriver.storage.googleapis.com/113.0.5672.24/chromedriver_linux64.zip
!unzip chromedriver_linux64
!rm -rf chromedriver_linux64.zip
!rm -rf LICENSE.chromedriver

--2023-05-31 14:59:01--  https://chromedriver.storage.googleapis.com/113.0.5672.24/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 173.194.210.128, 2607:f8b0:400c:c0f::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7172785 (6.8M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   6.84M  --.-KB/s    in 0.06s   

2023-05-31 14:59:01 (107 MB/s) - ‘chromedriver_linux64.zip’ saved [7172785/7172785]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
  inflating: LICENSE.chromedriver    


##PAGINA A INSPECCIONAR

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')

# Ruta del controlador de Chrome
chrome_driver_path = '/content/chromedriver'

# Crear un objeto Service para especificar la ruta del controlador
service = Service(chrome_driver_path)

# Crear una instancia del navegador Chrome
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navegar a una página web
driver.get('https://tablademareas.com/cl/los-lagos/puerto-montt')
#driver.get('https://tablademareas.com/cl/magallanes/punta-arenas')

# Obtener el HTML de la página después de activar el evento onclick
html = driver.page_source

# Crear un objeto BeautifulSoup para analizar el HTML
soup = BeautifulSoup(html, 'html.parser')

# Navegar a una página web
driver.get('https://tablademareas.com/cl/los-lagos/puerto-montt')
#driver.get('https://tablademareas.com/cl/magallanes/punta-arenas')

wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.ID, 'selector_api_marine')))

# Activar el evento onclick del elemento
driver.execute_script("arguments[0].click();", element)

# Obtener el HTML de la página después de activar el evento onclick
html1 = driver.page_source

# Crear un objeto BeautifulSoup para analizar el HTML
soup1 = BeautifulSoup(html1, 'html.parser')

##DATOS CLIMATOLOGICOS

In [5]:
def buscar_elemento(soup, tag, identificador):
    elemento = soup.find(tag, id=identificador) or soup.find(tag, class_=identificador)
    if elemento:
        return elemento.text.strip()
    else:
        return f'{tag.capitalize()} no encontrado'

In [6]:
## Función para buscar color en Descenso /Estable/ En Ascenso
def buscar_elemento_v(soup, tag, identificador):
    elemento = soup.find(tag, id=identificador) or soup.find(tag, class_=identificador)
    if elemento:
        return elemento
    else:
        return None

In [7]:
## Retorno de estado_color hacia estado
def obtener_estado(Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color):
    if Presion_Actividad_Pesca == 'ascenso_color':
        resultado = 'en ascenso'
    elif Presion_Actividad_Pesca == 'estable_color':
        resultado = 'estable'
    elif Presion_Actividad_Pesca == 'descenso_color':
        resultado = 'en descenso'
    else:
        resultado = "Presión de actividad de pesca desconocida"

    return resultado

In [8]:
## Retorno mensaje de pesca
def obtener_mensaje(Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color):
    Mensaje = [
        'Muy buena señal. Actividad alta prevista',
        'Actividad normal de los peces',
        'Buena pesca al principio, pero pronto dejarán de alimentarse'
    ]

    if Presion_Actividad_Pesca == 'ascenso_color':
        resultado = Mensaje[0]
    elif Presion_Actividad_Pesca == 'estable_color':
        resultado = Mensaje[1]
    elif Presion_Actividad_Pesca == 'descenso_color':
        resultado = Mensaje[2]
    else:
        resultado = "Presión de actividad de pesca desconocida"

    return resultado

In [9]:
## Retorno del valor activo Descenso /Estable/ En Ascenso
def encontrar_activa_v2(x, y, z):
    variables = {'ascenso_color': ascenso_color, 'estable_color': estable_color, 'descenso_color': descenso_color}
    activas = [variable for variable, valor in variables.items() if valor != '#ffffff']
    if len(activas) == 1:
        return activas[0]
    else:
        return "Ninguna o multiples variables activas"

In [10]:
## Estado para la pesca
def obtener_estado_activo_pesca(soup):
    elementos = soup.find_all('div', class_='circulo_estado_grafico_barometro1')
    
    for elemento in elementos:
        if 'circulo_estado_grafico_barometro1_activo' in elemento['class']:
            return elemento.next_sibling.strip()
    
    return "Estado pesca actual no encontrado"

In [11]:
#Viento
Viento = buscar_elemento(soup, 'span', 'velocidad_grafico_viento')
Viento_unidades = buscar_elemento(soup, 'span', 'unidades_grafico_viento')
Rachas = buscar_elemento(soup, 'span', 'rafagas_datos_grafico_tiempo_valor')
print('Viento:',Viento)
print('Unidades de viento:',Viento_unidades,)
print('Procedencia_Viento:',Rachas)
print('\n')

#Section Temperatura
Temperatura = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro')
Temperatura_Unidad = '°C'
Temperatura_Max = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro_max')
Temperatura_Unidad_Max = '°C'
Temperatura_Min = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro_min')
Temperatura_Unidad_Min = '°C'
Temperatura_Sensacion = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro_sensacion')
Temperatura_Unidad_Sensacion = '°C'

print('Temperatura:',Temperatura)
print('Unidad de Temperatura:',Temperatura_Unidad)
print('Temperatura Max:',Temperatura_Max)
print('Unidad de Temperatura Max:',Temperatura_Unidad_Max)
print('Temperatura Min:',Temperatura_Min)
print('Unidad de Temperatura Min:',Temperatura_Unidad_Min)
print('Temperatura Sensacion:',Temperatura_Sensacion)
print('Unidad de Temperatura Sensacion:',Temperatura_Unidad_Sensacion)


print('\n')

# Humedad
Humedad = buscar_elemento(soup, 'span', 'humedad_grafico_humedad')
Unidad_humedad = '%'
Pto_Rocio = buscar_elemento(soup, 'span', 'rocio_datos_grafico_tiempo_valor')
Unidad_Pto_Rocio = '°C'

print('Humedad:',Humedad)
print('Unidad Humedad:',Unidad_humedad)
print('Punto de Rocio:',Pto_Rocio)
print('Unidad Punto de Rocio:',Unidad_Pto_Rocio)
print('\n')

#Tiempo
Visibilidad = buscar_elemento(soup, 'span', 'numero_visibilidad_grafico_visibilidad')
Visibilidad_Unidad = buscar_elemento(soup, 'span', 'unidades_grafico_viento')

print('Visibilidad:',Visibilidad)
print('Visibilidad Unidad:',Visibilidad_Unidad)
print('\n')

#Presion
Presion = buscar_elemento(soup, 'span', 'numeros_presion_grafico_barometro')
Unidad_Presion = buscar_elemento(soup, 'span', 'unidades_presion_grafico_barometro')

print('Presion:',Presion)
print('Unidad Presion:',Unidad_Presion)
print('\n')

## Presión en la actividad pesquera

# Obtener el color de fondo
ascenso_elemento = buscar_elemento_v(soup, 'div', 'leyenda_influencia_grafico_barometro_ascenso')
estable_elemento = buscar_elemento_v(soup, 'div', 'leyenda_influencia_grafico_barometro_estable')
descenso_elemento = buscar_elemento_v(soup, 'div', 'leyenda_influencia_grafico_barometro_descenso')

# Obtener el color de fondo
ascenso_color = ascenso_elemento['style'].split(':')[1].strip()
estable_color = estable_elemento['style'].split(':')[1].strip()
descenso_color = descenso_elemento['style'].split(':')[1].strip()

Color_Presion_Actividad_Pesca=encontrar_activa_v2(ascenso_color,estable_color,descenso_color)     
Presion_Actividad_Pesca=obtener_estado(Color_Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color)
Mensaje_Presion_Actividad_Pesca = obtener_mensaje(Color_Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color)
Estado_activo_pesca=obtener_estado_activo_pesca(soup)

print('Color Actividad Pesca:',Color_Presion_Actividad_Pesca)
print('Presion Actividad Pesca:',Presion_Actividad_Pesca)
print('Mensaje Presion Actividad Pesca:',Mensaje_Presion_Actividad_Pesca)
print('Estado activo pesca:',Estado_activo_pesca)

print('\n')


Viento: 10
Unidades de viento: km/h
Procedencia_Viento: 13 km/h


Temperatura: 11
Unidad de Temperatura: °C
Temperatura Max: 13
Unidad de Temperatura Max: °C
Temperatura Min: 8
Unidad de Temperatura Min: °C
Temperatura Sensacion: 10
Unidad de Temperatura Sensacion: °C


Humedad: 96
Unidad Humedad: %
Punto de Rocio: 11
Unidad Punto de Rocio: °C


Visibilidad: 15
Visibilidad Unidad: km/h


Presion: 1019
Unidad Presion: hPa


Color Actividad Pesca: estable_color
Presion Actividad Pesca: estable
Mensaje Presion Actividad Pesca: Actividad normal de los peces
Estado activo pesca: MUY BUENO




## TABLAS MAREAS

In [12]:
def tabla_zona_costa_captura_datos(soup, id_fila):
    tabla = soup.find('div', id=f'tabla_datos_tiempo_datos_datos_cuadro_pie_{id_fila}')
    numero_lluvia = None
    cantidad_lluvia = None
    
    if tabla is not None:
        numero_lluvia_element = tabla.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_pie_color_lila')
        cantidad_lluvia_element = tabla.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_pie_color_celeste')
        
        if numero_lluvia_element is not None:
            numero_lluvia = numero_lluvia_element.text
        
        if cantidad_lluvia_element is not None:
            cantidad_lluvia = cantidad_lluvia_element.text

    return numero_lluvia, cantidad_lluvia

In [13]:
def tabla_zona_api_marine(soup1, id_fila):
  # Buscar el elemento padre por su ID
  etiqueta_padre_marine = soup.find('div', id='selector_api')

  # Activar estado de zona de costa
  etiqueta_zona_costa_marine = etiqueta_padre_marine.find('div', id='selector_api_marine')

  # Asignar el nuevo valor al atributo onclick
  etiqueta_zona_costa_marine['onclick'] = "CambiarAPI('marine', 1);"

  # Obtener el contenido principal de zona costera
  contenido_zona_costa_marine = etiqueta_zona_costa_marine.find('div', class_='selector_api_texto').text

  # Imprimir el resultado
  print(contenido_zona_costa_marine)

  contenedor_principal = soup1.find('div', id='tabla_datos_tiempo_datos')
  fila_datos = contenedor_principal.find('div', id=f'tabla_datos_tiempo_datos_{id_fila}')
  tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_tiempo').text
  datos_tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_datos')
  cuadros = datos_tiempo.find_all('div', class_='tabla_datos_tiempo_datos_datos_cuadro')
  resultados = []

  humedad_encontrada = False  # Variable para verificar si se ha encontrado una entrada de humedad válida

  for cuadro in cuadros:
      titulo_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_titulo')
      numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num')
      unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')

      titulo = titulo_element.text if titulo_element else ''
      numero = numero_element.text if numero_element else ''
      unidades = unidades_element.text if unidades_element else ''

      # Omitir los resultados con título vacío
      if not titulo:
          continue

      # Solución para capturar el valor de temperatura correctamente
      if titulo == 'Temp.':
          numero_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande_color')
          numero = numero_element.text if numero_element else ''

      # Solución para capturar el valor de viento correctamente
      if titulo == 'Viento':
          viento_valor_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
          viento_valor = viento_valor_element.text if viento_valor_element else ''
          resultado = {
              'titulo': titulo,
              'numero': viento_valor,
              'unidades': ''
          }
          resultados.append(resultado)
          continue

      resultado = {
          'titulo': titulo,
          'numero': numero,
          'unidades': unidades
      }
      
      if titulo == 'Humedad' and not humedad_encontrada:
          humedad_numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande')
          humedad_numero = humedad_numero_element.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_numero').text
          humedad_unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')
          humedad_unidades = humedad_unidades_element.text if humedad_unidades_element else ''
          if humedad_numero and humedad_unidades:
              resultado_humedad = {
                  'titulo': titulo,
                  'numero': humedad_numero,
                  'unidades': humedad_unidades
              }
              resultados.append(resultado_humedad)
              humedad_encontrada = True  # Marcar la humedad como encontrada
      else:
          resultados.append(resultado)

  presion_influencia_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_influencia')
  if presion_influencia_element is not None:
      presion_influencia = presion_influencia_element.text.strip()
      resultados.append({'Presión Influencia': presion_influencia})

  presion_estado_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_estado')
  if presion_estado_element is not None:
      presion_estado = presion_estado_element.text.strip()
      resultados.append({'Presión Estado': presion_estado})

  sensacion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo')
  if sensacion_element is not None:
      sensacion_valor_element = sensacion_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
      if sensacion_valor_element is not None:
          sensacion_valor = sensacion_valor_element.text.strip()
          resultados.append({'Sensación': sensacion_valor})

  punto_rocio_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo', string='punto rocío')
  if punto_rocio_element is not None:
      punto_rocio_valor_element = punto_rocio_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
      if punto_rocio_valor_element is not None:
          punto_rocio_valor = punto_rocio_valor_element.text.strip()
          resultados.append({'Punto Rocío': punto_rocio_valor})

  viento_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande', title='Velocidad del viento (km/h)\ny procedencia del viento')
  if viento_element is not None:
      viento_valor_element = viento_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
      if viento_valor_element is not None:
          viento_valor = viento_valor_element.text.strip()
          resultados.append({'Viento': viento_valor})

  direccion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_direccion')
  if direccion_element is not None:
      direccion_texto = direccion_element.text.strip()
      viento_resultado = next((r for r in resultados if r['titulo'] == 'Viento'), None)
      if viento_resultado:
          viento_resultado['unidades'] = direccion_texto

  resultados = [r for r in resultados if r.get('titulo') != 'Humidity']
  
  hora_element = fila_datos.find('div', class_='tabla_datos_tiempo_datos_tiempo_hora')
  numero_hora = hora_element.find('span', class_='tabla_datos_tiempo_datos_tiempo_hora_num').text
  letra_hora = hora_element.text.strip()[-1]
  
  numero_lluvia, cantidad_lluvia = tabla_zona_costa_captura_datos(soup, id_fila)
  
  resultados.append({'Hora': f'{numero_hora} {letra_hora}'})
  resultados.append({'Probabilidad de lluvia': numero_lluvia})
  resultados.append({'Cantidad de agua': cantidad_lluvia})
  
  return resultados

In [14]:
def tabla_zona_api_weather(soup, id_fila):
# Buscar el elemento padre por su ID
  etiqueta_padre_marine = soup.find('div', id='selector_api')

  # Activar estado de zona de costa
  etiqueta_zona_costa_marine = etiqueta_padre_marine.find('div', id='selector_api_weather')

  # Asignar el nuevo valor al atributo onclick
  etiqueta_zona_costa_marine['onclick'] = "CambiarAPI('weather', 1);"

  # Obtener el contenido principal de zona costera
  contenido_zona_costa_marine = etiqueta_zona_costa_marine.find('div', class_='selector_api_texto').text

  # Imprimir el resultado
  print(contenido_zona_costa_marine)

  contenedor_principal = soup.find('div', id='tabla_datos_tiempo_datos')
  fila_datos = contenedor_principal.find('div', id=f'tabla_datos_tiempo_datos_{id_fila}')
  tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_tiempo').text
  datos_tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_datos')
  cuadros = datos_tiempo.find_all('div', class_='tabla_datos_tiempo_datos_datos_cuadro')
  resultados = []

  humedad_encontrada = False  # Variable para verificar si se ha encontrado una entrada de humedad válida

  for cuadro in cuadros:
      titulo_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_titulo')
      numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num')
      unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')

      titulo = titulo_element.text if titulo_element else ''
      numero = numero_element.text if numero_element else ''
      unidades = unidades_element.text if unidades_element else ''

      # Omitir los resultados con título vacío
      if not titulo:
          continue

      # Solución para capturar el valor de temperatura correctamente
      if titulo == 'Temp.':
          numero_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande_color')
          numero = numero_element.text if numero_element else ''

      # Solución para capturar el valor de viento correctamente
      if titulo == 'Viento':
          viento_valor_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
          viento_valor = viento_valor_element.text if viento_valor_element else ''
          resultado = {
              'titulo': titulo,
              'numero': viento_valor,
              'unidades': ''
          }
          resultados.append(resultado)
          continue

      resultado = {
          'titulo': titulo,
          'numero': numero,
          'unidades': unidades
      }
      
      if titulo == 'Humedad' and not humedad_encontrada:
          humedad_numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande')
          humedad_numero = humedad_numero_element.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_numero').text
          humedad_unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')
          humedad_unidades = humedad_unidades_element.text if humedad_unidades_element else ''
          if humedad_numero and humedad_unidades:
              resultado_humedad = {
                  'titulo': titulo,
                  'numero': humedad_numero,
                  'unidades': humedad_unidades
              }
              resultados.append(resultado_humedad)
              humedad_encontrada = True  # Marcar la humedad como encontrada
      else:
          resultados.append(resultado)

  presion_influencia_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_influencia')
  if presion_influencia_element is not None:
      presion_influencia = presion_influencia_element.text.strip()
      resultados.append({'Presión Influencia': presion_influencia})

  presion_estado_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_estado')
  if presion_estado_element is not None:
      presion_estado = presion_estado_element.text.strip()
      resultados.append({'Presión Estado': presion_estado})

  sensacion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo')
  if sensacion_element is not None:
      sensacion_valor_element = sensacion_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
      if sensacion_valor_element is not None:
          sensacion_valor = sensacion_valor_element.text.strip()
          resultados.append({'Sensación': sensacion_valor})

  punto_rocio_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo', string='punto rocío')
  if punto_rocio_element is not None:
      punto_rocio_valor_element = punto_rocio_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
      if punto_rocio_valor_element is not None:
          punto_rocio_valor = punto_rocio_valor_element.text.strip()
          resultados.append({'Punto Rocío': punto_rocio_valor})

  viento_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande', title='Velocidad del viento (km/h)\ny procedencia del viento')
  if viento_element is not None:
      viento_valor_element = viento_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
      if viento_valor_element is not None:
          viento_valor = viento_valor_element.text.strip()
          resultados.append({'Viento': viento_valor})

  direccion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_direccion')
  if direccion_element is not None:
      direccion_texto = direccion_element.text.strip()
      viento_resultado = next((r for r in resultados if r['titulo'] == 'Viento'), None)
      if viento_resultado:
          viento_resultado['unidades'] = direccion_texto

  resultados = [r for r in resultados if r.get('titulo') != 'Humidity']
  
  hora_element = fila_datos.find('div', class_='tabla_datos_tiempo_datos_tiempo_hora')
  numero_hora = hora_element.find('span', class_='tabla_datos_tiempo_datos_tiempo_hora_num').text
  letra_hora = hora_element.text.strip()[-1]
  
  numero_lluvia, cantidad_lluvia = tabla_zona_costa_captura_datos(soup, id_fila)
  
  resultados.append({'Hora': f'{numero_hora} {letra_hora}'})
  resultados.append({'Probabilidad de lluvia': numero_lluvia})
  resultados.append({'Cantidad de agua': cantidad_lluvia})
  
  return resultados

In [15]:
def imprimir_tablas(soup):
  for id_fila in range(24):
    if (soup is globals().get('soup')):
      resultados = tabla_zona_api_weather(soup, id_fila)
      print(resultados)
    elif (soup is globals().get('soup1')):
      resultados = tabla_zona_api_marine(soup, id_fila)
      print(resultados)

In [16]:
imprimir_tablas(soup)


 Previsión en zona costera 
 COSTA DE PUERTO MONTT 

[{'titulo': 'Presión', 'numero': '1014', 'unidades': ''}, {'titulo': 'Temp.', 'numero': '11°', 'unidades': ' °C'}, {'titulo': 'Humedad', 'numero': '92', 'unidades': ' %'}, {'titulo': 'Viento', 'numero': '19', 'unidades': 'N'}, {'Presión Influencia': 'estable'}, {'Presión Estado': 'Muy bueno'}, {'Sensación': '8 °C'}, {'Punto Rocío': '9 °C'}, {'Hora': '0 h'}, {'Probabilidad de lluvia': '79% '}, {'Cantidad de agua': '0.5'}]

 Previsión en zona costera 
 COSTA DE PUERTO MONTT 

[{'titulo': 'Presión', 'numero': '1014', 'unidades': ''}, {'titulo': 'Temp.', 'numero': '11°', 'unidades': ' °C'}, {'titulo': 'Humedad', 'numero': '93', 'unidades': ' %'}, {'titulo': 'Viento', 'numero': '19', 'unidades': 'N'}, {'Presión Influencia': 'estable'}, {'Presión Estado': 'Muy bueno'}, {'Sensación': '8 °C'}, {'Punto Rocío': '9 °C'}, {'Hora': '1 h'}, {'Probabilidad de lluvia': '87% '}, {'Cantidad de agua': '0.5'}]

 Previsión en zona costera 
 COSTA DE PUE

In [17]:
imprimir_tablas(soup1)


 Previsión en aguas abiertas 
 MAR EN PUERTO MONTT 

[{'titulo': 'Presión', 'numero': '1013', 'unidades': ''}, {'titulo': 'Temp.', 'numero': '11°', 'unidades': ' °C'}, {'titulo': 'Humedad', 'numero': '92', 'unidades': ' %'}, {'titulo': 'Viento', 'numero': '42', 'unidades': 'N'}, {'Presión Influencia': 'en ascenso'}, {'Presión Estado': 'Muy bueno'}, {'Sensación': '8 °C'}, {'Punto Rocío': '10 °C'}, {'Hora': '0 h'}, {'Probabilidad de lluvia': '79% '}, {'Cantidad de agua': '0.5'}]

 Previsión en aguas abiertas 
 MAR EN PUERTO MONTT 

[{'titulo': 'Presión', 'numero': '1014', 'unidades': ''}, {'titulo': 'Temp.', 'numero': '11°', 'unidades': ' °C'}, {'titulo': 'Humedad', 'numero': '92', 'unidades': ' %'}, {'titulo': 'Viento', 'numero': '40', 'unidades': 'N'}, {'Presión Influencia': 'en ascenso'}, {'Presión Estado': 'Muy bueno'}, {'Sensación': '8 °C'}, {'Punto Rocío': '10 °C'}, {'Hora': '1 h'}, {'Probabilidad de lluvia': '87% '}, {'Cantidad de agua': '0.5'}]

 Previsión en aguas abiertas 
 MA

## INDICE ULTRAVIOLETA

In [18]:
def mensaje_de_seguridad(uv_maximo_img_num):
   Mensaje = ['Puedes salir sin necesidad de tomar ninguna medida de protección solar.', 'Viste camiseta, gorra y gafas. \n Usa protección solar 30+. \n Mantente a la sombra en las horas centrales del día.', 'Viste camiseta, gorra y gafas. \n Usa protección solar 50+. \n Mantente a la sombra todo el tiempo posible y evita salir en las horas centrales del día.']
   if uv_maximo_img_num <3:
    resultado = Mensaje[0]
   elif (uv_maximo_img_num >3) & (uv_maximo_img_num <8) :
    resultado = Mensaje[1]
   elif uv_maximo_img_num >=8:
    resultado = Mensaje[2]
   else:
    resultado = "Recomendacion para indice ultravioleta no encontrado"

   return resultado


In [19]:
uv_maximo_texto = soup.find('div', {'id': 'uv_maximo_texto'}).text

# Obtener el número 2 del div con id="uv_maximo_img_num"
uv_maximo_img_num = soup.find('div', {'id': 'uv_maximo_img_num'}).text

# Imprimir los resultados
print('Indice ultravioleta:', uv_maximo_texto)
print('Valor de indice ultravioleta:', uv_maximo_img_num)
print('Mensaje respecto al indice ultravioleta:',mensaje_de_seguridad(int(uv_maximo_img_num)))


Indice ultravioleta: moderado
Valor de indice ultravioleta: 3
Mensaje respecto al indice ultravioleta: Recomendacion para indice ultravioleta no encontrado


## TEMPERATURA DEL AGUA

In [20]:
actual_element = soup.find('span', {'id': 'c_grafico_temp_agua_txt_grados_agua_actual'})

# Extraer el valor del elemento actual
temp_actual_agua = actual_element.text.strip()

# Encontrar el elemento con id "c_grafico_temp_agua_txt_grados_agua_media"
media_element = soup.find('span', {'id': 'c_grafico_temp_agua_txt_grados_agua_media'})

# Extraer el valor del elemento de la media
temp_media_agua = media_element.text.strip()

# Imprimir los valores
print("Temperatura actual del agua:", temp_actual_agua)
print("Temperatura media del agua:", temp_media_agua)

Temperatura actual del agua: 11°C
Temperatura media del agua: 10°C


In [21]:
rafificacion_promedio=0
valores_horarios_valor = []
valores_horarios = []
horas = soup.find_all('div', {'class': 'evolucion_temp_agua_hora'})
for i, hora in zip(range(14),horas):
  id_valor_relleno = f"evolucion_temp_agua_relleno_cuadro_{i}"
  valor_incremental = soup.find('div', {'class': 'evolucion_temp_agua_relleno_cuadro', 'id': id_valor_relleno}).text.strip()
  valor_horario = hora.text.strip()
  valores_horarios.append(valor_horario)
  valores_horarios_valor.append(valor_incremental)
for valor, valor1 in zip(valores_horarios_valor,valores_horarios):
  print(valor1, valor)
  numero = ''.join(filter(str.isdigit, valor))
  rafificacion_promedio=rafificacion_promedio+int(numero)
print(rafificacion_promedio/13)

0 h 8°
1 h 11°
2 h 11°
3 h 8°
4 h 11°
5 h 11°
6 h 8°
7 h 11°
8 h 11°
9 h 8°
10 h 11°
11 h 11°
12 h 8°
13 h 11°
10.692307692307692


## OLEAJE

In [24]:
valor_fecha = soup.find('div', class_='texto_grafico_estado_actual_fecha').text.strip()
print("Valor de la fecha:", valor_fecha)

direccion_de_oleaje = soup.find('span', id='swellDir16Point').text
print("Direccion de oleaje:", direccion_de_oleaje)

valor_de_oleaje = soup.find('span', id='swellDir').text
print("Valor de oleaje:", valor_de_oleaje)

altura_significativa = soup.find('span', id='sigHeight_m').text
print("Altura significativa de las olas:", altura_significativa)

periodo_olas = soup.find('span', id='swellPeriod_secs').text
print("Periodo de las olas:", periodo_olas)

valor_costa = soup.find('span', class_='grafico_viento_oleaje_actual_titulo_txt_costa').text.strip()
valor_direccion = soup.find('span', id='oleaje_viento_direccion_txt').text.strip()
valor_grados = soup.find('span', id='oleaje_viento_direccion_grados').text.strip()
valor_velocidad = soup.find('span', id='oleaje_viento_velocidad').text.strip()

print("Valor de costa:", valor_costa)
print("Valor de dirección:", valor_direccion)
print("Valor de grados:", valor_grados)
print("Valor de velocidad:", valor_velocidad)

valor_aguas_abiertas = soup.find('span', class_='grafico_viento_oleaje_actual_titulo_txt_mar').text.strip()
valor_direccion = soup.find('span', id='oleaje_viento_marine_direccion_txt').text.strip()
valor_grados = soup.find('span', id='oleaje_viento_marine_direccion_grados').text.strip()
valor_velocidad = soup.find('span', id='oleaje_viento_marine_velocidad').text.strip()

print("Valor de aguas abiertas:", valor_aguas_abiertas)
print("Valor de dirección:", valor_direccion)
print("Valor de grados:", valor_grados)
print("Valor de velocidad:", valor_velocidad)


alturas_oleaje_actual = soup.find_all('div', class_='alturas_oleaje_actual')
for elemento in alturas_oleaje_actual:
    titulo = elemento.find('div', class_='alturas_oleaje_actual_titulo').text.strip()
    texto = elemento.find('div', class_='alturas_oleaje_actual_texto').text.strip()
    valor = elemento.find('div', class_='circulo_alturas_oleaje').text.strip()
    print("Título:", titulo)
    print("Texto:", texto)
    print("Valor:", valor)


Valor de la fecha: 31 DE MAYO DE 2023, 10:59 h
Direccion de oleaje: NE
Valor de oleaje: 40
Altura significativa de las olas: 0.5 m
Periodo de las olas: 0.1 s
Valor de costa: costa
Valor de dirección: N
Valor de grados: 357
Valor de velocidad: 10 km/h
Valor de aguas abiertas: aguas abiertas
Valor de dirección: N
Valor de grados: 358
Valor de velocidad: 27 km/h
Título: Altura más frecuente
Texto: La mayoría de las olas tendrán la mitad de la altura significativa.
Valor: 0.3 m
Título: Altura significativa
Texto: Aprox. el 14% de las olas serán más grandes que la altura significativa (aprox. 1 de cada 7 olas).
Valor: 0.5 m
Título: Altura máxima
Texto: Lo normal es esperar una ola del doble de la altura significativa 3 veces cada 24 horas.
Valor: 1.0 m
